<a href="https://colab.research.google.com/github/TavionK/CPSC-419/blob/main/projects/CrateBarrel/MyCrateAndBarrel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crate and Barrel project
Objective: Create a classifier that predicts, based on the features, what product it is.

1) Import libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier

2) Import the data

In [2]:
final_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
crate_train, crate_test = train_test_split(train, test_size= 0.2)
crate_test.set_index('id', inplace=True)
crate_train.set_index('id', inplace=True)
final_test.set_index('id', inplace=True)

3) Create the features from the label

In [3]:
crate_train_features = crate_train.drop('target', axis=1)
crate_train_label = crate_train['target']
crate_test_features = crate_test.drop('target', axis=1)
crate_test_label = crate_test['target']

4) Try using various versions of a DecisionTree Classifier

In [82]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(crate_train_features, crate_train_label)
p = clf.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.7133968972204267

try using a bagging classifier

In [83]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, bootstrap=True, n_jobs=-1)
bagging_clf.fit(crate_train_features, crate_train_label)
p = bagging_clf.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.6277472527472527

Try using pasting

In [84]:
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(crate_train_features, crate_train_label)
p = pasting_clf.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.6149806076276665

Try using subspaces

In [85]:
subspace_clf = BaggingClassifier(clf, n_estimators=20, max_features=7, 
                                bootstrap=True, n_jobs=-1)
subspace_clf.fit(crate_train_features, crate_train_label)
p = subspace_clf.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.5939722042663219

Try using random patches

In [86]:
random_clf = BaggingClassifier(clf, n_estimators=100, max_features=7, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
random_clf.fit(crate_train_features, crate_train_label)
p = random_clf.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.5252100840336135

Try with different hyperparmeters

In [87]:
clf2 = tree.DecisionTreeClassifier(criterion='entropy')
first = BaggingClassifier(clf2, n_estimators=20, max_features=0.7, 
                                 max_samples=0.7, bootstrap=False, n_jobs=-1)
first.fit(crate_train_features, crate_train_label)
p = first.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.7956528765352294

5) Try using xgboost

In [88]:
!nvidia-smi

Mon May  3 00:53:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    30W /  70W |   7042MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [89]:
from xgboost import XGBClassifier
params1 = { 'objective':'multi:softprob', 'n_estimators': 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
model1 = XGBClassifier(**params1)

In [90]:
model1.fit(crate_train_features, crate_train_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='multi:softprob',
              predictor='gpu_predictor', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, tree_method='gpu_hist', verbosity=1)

In [92]:
p = model1.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.7957336780866192

Try using different parameters

In [10]:
params2 = { 'objective':'multi:softprob', 'n_estimators': 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'max_depth':4}
model2 = XGBClassifier(**params2)
model2.fit(crate_train_features, crate_train_label)
p = model2.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.7993697478991597

In [95]:
params3 = { 'objective':'multi:softprob', 'n_estimators': 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'max_depth':5}
model3 = XGBClassifier(**params3)
model3.fit(crate_train_features, crate_train_label)
p = model3.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.8103587588881707

In [8]:
params4 = { 'objective':'multi:softprob', 'n_estimators': 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'max_depth':10, 'learning_rate':0.30}
model4 = XGBClassifier(**params4)
model4.fit(crate_train_features, crate_train_label)
p = model4.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.8190853264382676

In [4]:
params5 = { 'objective':'multi:softprob', 'n_estimators': 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'max_depth':20, 'learning_rate':0.30}
model5 = XGBClassifier(**params5)
model5.fit(crate_train_features, crate_train_label)
p = model5.predict(crate_test_features)
accuracy_score(crate_test_label, p)

0.8218325791855203

6) Appy it to the test data and get the csv file

In [5]:
final_test

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,3,0,0,0,3,2,1,0,0,0,0,0,0,0,5,3,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,20,0,0,0,0,0
2,2,2,14,16,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,2,0,4,0,4,0,0,0,0,2,0,0,0,8,0,0,0,0,0,...,24,0,0,0,0,0,0,0,0,0,6,8,0,0,0,0,0,0,0,0,0,0,0,2,0,0,4,0,2,0,0,0,0,0,0,4,0,0,2,0
3,0,1,12,1,0,0,0,0,0,0,7,1,0,0,0,7,0,2,0,0,0,4,0,0,0,1,1,2,0,0,0,0,0,0,1,0,0,2,0,0,...,10,1,0,2,0,0,1,6,1,1,0,0,1,1,1,2,0,0,2,0,0,0,0,0,0,0,6,0,2,0,0,0,0,0,2,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,3,0,0,0,0,0,0,0,0,4,0,0,0,1,0,0,0,2,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,9,0,0,2,0,0,0,0,6,0,8,0,0,0,0,1,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0
5,1,0,0,1,0,0,1,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,4,0,5,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144364,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,5,0,0,1,0,1,0,2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,...,0,2,0,0,0,0,0,0,0,0,2,0,0,9,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,2,1,1,0,0,0,0,0
144365,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,1,11,0,0,0,0,0
144366,0,1,0,0,0,0,1,1,0,0,0,0,0,7,0,1,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,0,1,0,1,0,3,1,0,6,...,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,5,0,0,1,2,0,0,0,0,0,0,0,0,0,0,1,3,1,1,0,0,1,0,0


In [12]:
final = model2.predict_proba(final_test)

df = pd.DataFrame(final)
df.index += 1 
df.index.name = 'id'
df.columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']
df
df.to_csv('results5.csv')